In [16]:
import os, pandas as pd, numpy as np
import math
import time
path='d:/programming_files/omnivest/nba'
os.chdir(path)
#data=pd.read_csv('game_data.csv')
data=pd.read_pickle('data.pkl')
games=pd.read_pickle('games.pkl')
teams=pd.read_pickle('teams.pkl')
qt_data=pd.read_pickle('qt_data.pkl')

In [ ]:
#calculations that we don't need any more
data['diff']=data['away_score']-data['home_score']
data['sum']=data['away_score']+data['home_score']
games2=pd.read_pickle('games2.pkl')

teams=data['home_tm']
teams=teams.drop_duplicates()
teams=teams.values.tolist()
teams=pd.DataFrame(data={'name':teams})
teams_li=data['home_tm']
teams_li=teams_li.drop_duplicates()
teams_li=teams_li.values.tolist()

temp=[]
temp2=[]
for i in range(len(teams_li)):
    for j in range(len(teams_li)):
        if i!=j:
            temp.append(teams_li[i])
            temp2.append(teams_li[j])
games=pd.DataFrame(data={'a_tm':temp,'h_tm':temp2})

temp3=[]
games['ind']=games['a_tm']+games['h_tm']
games=games.set_index('ind')
games.index.name=None

qt_data=pd.read_csv('NBA_2014_2017_Data.csv')
qt_data['ind']=qt_data['away_team']+qt_data['home_team']
qt_data=qt_data.set_index('ind')
qt_data.index.name=None
qt_data.to_pickle('qt_data.pkl')

In [21]:
#useful calculations need to be done every time
c_home=-data['diff'].mean()
lmda=data['sum'].mean()
#\sigma^2_(\delta G)
diffvar=data['diff'].var()
#variance of q_ij
vqij=2*diffvar
N=1000

In [17]:
#team related functions
class Team:
    def __init__(self, name, data):
        self.name=name
        self.data=data
        #the lines that include team=name as 
        #the away/home team
        self.a_games=data.loc[data['away_tm']==name]
        self.h_games=data.loc[data['home_tm']==name]
        #average sum of scores for all games
        self.lmda=data['sum'].mean()
        self.sumvar=data['sum'].var()
        self.diffvar=data['diff'].var()
    #the average sum of scores scored and conceded by team=name
    def sscore(self):
        tp1=self.a_games['sum'].values.tolist()
        tp1.extend(self.h_games['sum'].values.tolist())
        tp2=np.mean(tp1)
        b_n=1/(1+3/(len(tp1)*self.sumvar))
        tp3=self.lmda+b_n*(tp2-self.lmda)
        return tp3
    #the average (scored-conceded) per game for team=name
    def difscore(self):
        tp1=self.a_games['diff'].values.tolist()
        tp1.extend([-x for x in self.h_games['diff'].values.tolist()])
        tp2=np.mean(tp1)
        a_n=1/(1+3/(len(tp1)*self.diffvar))
        tp3=a_n*tp2
        return tp3

In [22]:
#the \Delta G_i for each team
diff_li=[]
for row in teams['name']:
    temp1=Team(row,data)
    diff_li.append(temp1.difscore())
teams['dlt_g']=diff_li

#the \Sigma G_i for each team
sum_li=[]
for row in teams['name']:
    temp1=Team(row,data)
    sum_li.append(temp1.sscore())
teams['sigma_g']=sum_li

In [72]:
teams.to_pickle('teams.pkl')

In [36]:
games.to_pickle('games.pkl')

In [31]:
data.to_pickle('data.pkl')

In [23]:
#game related functions
#tm1 is away team, tm2 is home team
class Game:
    def __init__(self, tm1, tm2):
        self.tm1=tm1
        self.tm2=tm2
        #value of sum of goals g_i+g_j
        self.sgoals=teams.ix[self.tm1,'sigma_g']+teams.ix[self.tm2,'sigma_g']-lmda
        #mean value of q_ij considering home advantage
        #attention i is away team here
        self.mqij=teams.ix[self.tm1,'dlt_g']-teams.ix[self.tm2,'dlt_g']-c_home
        self.times=N
    #monte carlo simulation for poisson
    def sim_result(self):
        temp=0
        game_mqij=games['mqij'].loc[self.tm1+self.tm2]
        game_sgoals=games['sgoals'].loc[self.tm1+self.tm2]
        a_mean=(game_mqij+game_sgoals)/2
        h_mean=(game_sgoals-game_mqij)/2
        for i in range(self.times):
            t=0 
            I=0
            while t<=1:
                t=t-1/(a_mean+h_mean)*math.log(np.random.random_sample())
                I=I+1
            temp=temp+I
        g_i=round(temp/self.times*a_mean/(a_mean+h_mean))
        g_j=round(temp/self.times*h_mean/(a_mean+h_mean))
        return g_i, g_j

In [8]:
#Get for each game the g_i+g_j and the q_ij
temp1=[]
temp2=[]
for index, row in games.iterrows():
    one_game=Game(row['a_tm'],row['h_tm'])
    temp1.append(one_game.sgoals)
    temp2.append(one_game.mqij)
games['sgoals']=temp1
games['mqij']=temp2

In [24]:
#Get for each game the simulation result
def all_sim():
    t1=time.time()
    tp_1=[]
    tp_2=[]
    for index, row in games.iterrows():
        one_game=Game(row['a_tm'],row['h_tm'])
        rlt1, rlt2=one_game.sim_result()
        tp_1.append(rlt1)
        tp_2.append(rlt2)
    games['sim_ascore']=tp_1
    games['sim_hscore']=tp_2
    t2=time.time()-t1
    print('Time used for this simulation is '+str(t2)+' seconds')
    return 
all_sim()

In [35]:
#test the model
def test():
    temp1=0
    temp2=0
    for index, row in data.iterrows():
        real_mar=row['away_score']-row['home_score']
        sim_mar=games['sim_ascore'].loc[row['away_tm']+row['home_tm']]-games['sim_hscore'].loc[row['away_tm']+row['home_tm']]
        temp1=temp1+int(real_mar*sim_mar>0)
        temp2=temp2+int(math.fabs(real_mar-sim_mar)<=5)
    pick=temp1/len(data)*100
    mar=temp2/len(data)*100
    return pick,mar

pick_accu, mar_accu=test()
print('We pick the right winner for %s percent of time, predict the right margin for %s percent of time'
      %(pick_accu, mar_accu))

We pick the right winner for 63.92405063291139 percent of time, predict the right margin for 36.795166858457996 percent of time


The accuracy rate of nbagamesim.com is as follows:
Correct Picks 	48 (70.6%)
Final Margin within 5 Pts 	20 (29.4%)

Thus we have a better accuracy then them in predicting the scores
What's more, we have a bigger sample size, thus our result is much more impressive. Cheers.